# Baroclinic instability in the 2-layer QG model (Phillips problem)

In this notebook we illustrate some elementary aspects of the baroclinic instability in the simplest possible context, that of a two-layer quasi-geostrophic model (the so-called Phillips problem).

The notebook assumes a basic familiarity with the quasi-geostrophic equations and the linear stability problem. We give in the next a brief reminder of the main results without much details. Here only the qualitative aspects matter, the details of the formulae are not crucial. After this section we will provide some code to run numerical experiments which will allow to address the questions asked in the last three sections.

# The problem

This section is a brief reminder of the setup of the Phillips problem. For more information, you can refer to standard textbooks such as the ones of McWilliams, Vallis, or Cushman-Roisin.

## The model

We consider the two-layer quasi-geostrophic equations:
\begin{align*}
\frac{Dq_i}{Dt} &= 0, i=1,2,\\
q_1 &= \Delta \psi_1 +\beta y + k_d^2\frac{\psi_2-\psi_1}{2},\\
q_2 &= \Delta \psi_2 +\beta y + k_d^2\frac{\psi_1-\psi_2}{2},
\end{align*}

where $q$ is potential vorticity, $\psi$ the stream function, $\beta=\frac{df}{dy}$ the gradient of planetary vorticity, and $k_d$ the deformation wave number, inversely proportionnal to the Rossby deformation radius. This sytem can be interpreted as a discretized version of the continuously stratified QG equations, and the deformation wave number is related to the buoyancy frequency by $k_d^2=\frac{8f_0^2}{N^2H^2}$ and $H$ is the depth of each layer (we asumed they have the same).

We will consider the dynamics of small perturbations on top of a prescribed, meridionnally invariant, zonal flow: $\psi_i=\Psi_i+\psi_i'$, $q_i=Q_i+q_i'$ (we will drop the primes in the following), with $\Psi_1=-Uy$, $\Psi_2=Uy$, $Q_1=\beta y +k_d^{2}Uy$ and $Q_2=\beta y -k_d^{2}Uy$.

## Linear stability analysis

Linearizing the equations around the prescribed mean-flow and looking for solutions of the form $\psi_j'=\Re\lbrack \tilde{\psi}_j e^{ik(x-ct)+i\ell{}y}\rbrack$, we find after a bit of algebra that the condition for existence of non-trivial solutions leads to 
\begin{equation*}
  c = -\frac{\beta}{K^2+k_d^2} \left\lbrack 1+\frac{k_d^2}{2K^2} \pm \frac{k_d^2}{2K^2}\sqrt{1+\frac{4K^4(K^4-k_d^4)}{k_d^4k_\beta^4}} \right\rbrack,
\end{equation*}
with $K^2=k^2+\ell^2$ and $k_\beta=\sqrt{\beta/U}$.

- In the special case where $\beta=0$, the solution becomes
  \begin{equation*}
    c = \pm U \sqrt{\frac{K^2-k_d^2}{K^2+k_d^2}}.
  \end{equation*}
  Hence, modes with $K<k_d$ are unstable (i.e. with wavelength $\lambda>\lambda_c\equiv \frac{2\pi}{\sqrt{8}}L_d\approx 2.2L_d$ with $L_d$ the deformation radius), with a growth rate $\sigma=Uk\sqrt{\frac{k_d^2-K^2}{k_d^2+K^2}}$.
  For a given $K$ the most unstable modes do not vary in the meridional direction ($\ell=0$), and the maximum growth rate is attained for $k=\sqrt{\sqrt{2}-1}k_{d}\approx0.64k_d$, i.e. for a wave length $\lambda_m=\frac{2\pi}{\sqrt{8(\sqrt{2}-1)}}L_{d}\approx 3.5L_d$

- In the general case, if $k_d<k_\beta$, the amplitude of the perturbation does not increase or decrease, for any wave number $K$. In particular, in this case there is no unstable normal-mode solution. On the other hand, if $k_d > k_\beta$, modes with $\frac{k_d^4}{2}\left(1-\sqrt{1-\frac{k_\beta^4}{k_d^4}}\right)< K^4 < \frac{k_d^4}{2}\left(1+\sqrt{1-\frac{k_\beta^4}{k_d^4}}\right)$ are unstable. We note that there is now a small-wave number cutoff, which was not present when $\beta=0$. Hence the $\beta$-effect stabilizes the largest scales.
In addition, the condition $k_d>k_\beta$ means that the vertical shear $\Lambda = (U_1-U_2)/(H/2)$ needs to be larger than some critical value for instability to occur: $\Lambda > \frac{4\beta}{Hk_d^2}$. This is equivalent to requiring that the background PV gradient should change have a different sign in the two layers (one of the necessary conditions in the Charney-Stern-Pedlosky criterion). Again, this condition is not present when $\beta$ vanishes, because then the PV gradients are just $k_d^2U$ and $-k_d^2U$, respectively, which always have opposite signs. In the general case, the growth rate of unstable modes is again maximum for $\ell=0$ and is given by $\sigma=\frac{\beta{}kk_{d}^2}{2k^2(k^2+k_d^2)}\sqrt{\frac{4k^4(k_d^4-k^4)}{k_d^{4}k_\beta^4}-1}$, which can be written in nondimensional form: $\tilde{\sigma}=\frac{\sqrt{4\tilde{k}^4(1-\tilde{k}^4)-\tilde{k_\beta}^4}}{\tilde{k}(\tilde{k}^2+1)\tilde{k_\beta}^4}$ with $\tilde{k}=k/k_d$ and $\tilde{\sigma}=\sigma{}k_d/\beta$.

# Code

In this section we define the functions which will allow us to run the simulations and to make plots.

We will use the `dedalus` package, which is a general pseudo-spectral solver for partial differential equations.
Because the goal of the notebook is to explore the physics of the 2-layer model and in particular the baroclinic instability in a short amount of time, we essentially use dedalus as a black box, and do not pay much attentions to the numerical methods used to solve the equations.

First we import the modules we will need.

In [56]:
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['NUMEXPR_MAX_THREADS'] = '1'

In [23]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)
import h5py

Now we define the domain and numerical resolution that we will use here: a doubly periodic square domain with 64 grid points in each direction.

In [6]:
# Resolution
N = 64

# Domain
coords = d3.CartesianCoordinates('x', 'y')
dist = d3.Distributor(coords, dtype=np.float64)
xbasis = d3.RealFourier(coords['x'], size=N, bounds=(-np.pi, np.pi), dealias=3/2)
ybasis = d3.RealFourier(coords['y'], size=N, bounds=(-np.pi, np.pi), dealias=3/2)
bases = (xbasis, ybasis)

x, y = dist.local_grids(xbasis, ybasis)
ex, ey = coords.unit_vector_fields(dist)

To make it easier to run multiple simulations without having to copy-past the code each time, we define two functions: `build_problem` to build the numerical problem with the desired parameters, and `run_simulation` to run it with chosen numerical parameters (duration of the simulation, name of the output directory, output frequency). To run a simulation, you need to:
- call `build_problem` with the chosen physical parameters
- define the initial condition
- call `run_simulation`

When the simulation is finished, you can read the output file and make all the plots you want to analyse it.

In [199]:
def build_problem(beta=0, U=1, kd2=100, nu=1e-1, alpha=0):
    """
    Create a d3.core.problems.InitialValueProblem object for the two-layer QG model with background shear and return it.

    Keyword Arguments
    -----------------
    beta: float, the gradient of planetary vorticity, beta=2*Omega*cos(phi)/a.
    U: float, the background zonal wind
    kd2: int, the (square of) the deformation wave number
    nu: float, the hyperviscosity coefficient (we use third-order hyperviscosity)
    alpha: float, the bottom friction
    """
    # Define the derivative operators
    dx = lambda A:(d3.Differentiate(A, coords['x']))
    dy = lambda A:(d3.Differentiate(A, coords['y']))

    # Define the fields
    # For the upper-layer
    psi1 = dist.Field(name='psi1', bases=bases) #stream function
    q1 = dist.Field(name='q1', bases=bases) # potential vorticity
    tau_psi1 = dist.Field(name='tau_psi1') # constraint for gauge condition

    # For the lower-layer
    psi2 = dist.Field(name='psi2', bases=bases) # stream function
    q2 = dist.Field(name='q2', bases=bases) # potential vorticity
    tau_psi2 = dist.Field(name='tau_psi2') # constraint for gauge condition

    # Define the problem (equations)
    problem = d3.IVP([psi1, psi2, q1, q2, tau_psi1, tau_psi2], namespace=locals())
    problem.add_equation("dt(q1) + U*dx(q1) + (beta + kd2*U)*dx(psi1) - nu*lap(lap(lap(q1))) = -dx(psi1)*dy(q1) + dy(psi1)*dx(q1)")
    problem.add_equation("dt(q2) - U*dx(q2) + (beta - kd2*U)*dx(psi2) - nu*lap(lap(lap(q2))) + alpha*lap(psi2) = -dx(psi2)*dy(q2) + dy(psi2)*dx(q2)")
    problem.add_equation("integ(psi1) = 0") # gauge condition, otherwise the problem is ill-defined
    problem.add_equation("integ(psi2) = 0") # gauge condition, otherwise the problem is ill-defined
    problem.add_equation("q1 - lap(psi1) - (kd2/2)*(psi2-psi1) + tau_psi1 = 0")
    problem.add_equation("q2 - lap(psi2) - (kd2/2)*(psi1-psi2) + tau_psi2 = 0")

    return problem

def run_simulation(name, problem, stop_sim_time = 5, initial_timestep = 1e-3, snap_dt=1e-1):
    """
    Run a simulation with dedalus and create output files.

    Parameters
    ----------
    name: str, the name of the simulation, used for the directory containing output files
    problem: d3.core.problems.InitialValueProblem object, the problem to solve

    Keyword Arguments
    -----------------
    stop_sim_time: float, the total duration of the simulation (default 5.0)
    initial_timestep: float, an initial value for the timestep, overridden by the adapative time step in subsequent steps (default 1e-3)
    snap_dt: float, the output frequency (default 0.1). Decrease it if necessary to keep output files of reasonable size.
    """
    # Define the time-stepping method
    timestepper = d3.RK443
    # Build the solver
    solver = problem.build_solver(timestepper)
    solver.stop_sim_time = stop_sim_time

    # Recover the fields for later use
    psi1 = problem.namespace['psi1']
    psi2 = problem.namespace['psi2']
    q1 = problem.namespace['q1']
    q2 = problem.namespace['q2']
    
    # Configure adaptive time-stepping
    CFL = d3.CFL(solver, initial_dt=initial_timestep, max_dt=1e-1)
    dx = lambda A:(d3.Differentiate(A, coords['x']))
    dy = lambda A:(d3.Differentiate(A, coords['y']))
    CFL.add_velocity(-dy(psi1)*ex + dx(psi1)*ey)

    # Configure the data to store in output files
    snapshots = solver.evaluator.add_file_handler(name, sim_dt=snap_dt)
    snapshots.add_task(psi1, name='psi1')
    snapshots.add_task(q1, name='vorticity1')
    snapshots.add_task(psi2, name='psi2')
    snapshots.add_task(q2, name='vorticity2')
    snapshots.add_task(d3.Integrate((dx(psi1))**2 + (dy(psi1))**2, ('x', 'y')), name='KE1')
    snapshots.add_task(d3.Integrate((dx(psi2))**2 + (dy(psi2))**2, ('x', 'y')), name='KE2')

    # Actually run the simulation
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if solver.iteration % 100 == 0:
            logger.info('Iteration=%i, Time=%e, dt=%e' %(solver.iteration, solver.sim_time, timestep))

    # Post-processing: add parameter values to output files
    with h5py.File(f"{name}/{name}_s1.h5", 'r+') as datafile:
        datafile.attrs['mean_flow'] = problem.namespace['U']
        datafile.attrs['beta'] = problem.namespace['beta']
        datafile.attrs['kd'] = np.sqrt(problem.namespace['kd2'])
        datafile.attrs['nu'] = problem.namespace['nu']
        datafile.attrs['alpha'] = problem.namespace['alpha']

def monochromatic_wave(k, l, ampl=1, phase=0):
    """
    Return a 2D monochromatic wave with given parameters.

    Parameters
    ----------
    k: int, the zonal wave number
    l: int, the meridional wave number

    Keyword Arguments
    -----------------
    ampl: float, the amplitude of the wave
    phase: float, the phase of the wave
    """
    x, y = dist.local_grids(xbasis, ybasis)
    return  ampl*np.cos(k*x + l*y+phase)

def pv_from_streamfunction(psi1, psi2, kd2):
    """
    Compute the potential vorticity from the stream function in the two layers and return it.

    Parameters
    ----------
    psi1: d3.core.field.Field, the upper-layer stream function
    psi2: d3.core.field.Field, the lower-layer stream function
    kd2: int, the (square of) the deformation wave number
    """
    zeta1 = d3.Laplacian(psi1).evaluate()
    zeta2 = d3.Laplacian(psi2).evaluate()
    for field in (psi1, psi2, zeta1, zeta2):
        field.change_scales(1)
    q1 = zeta1['g'] + (kd2/2)*(psi2['g']-psi1['g'])
    q2 = zeta2['g'] - (kd2/2)*(psi2['g']-psi1['g'])
    return q1, q2

Next we define functions to create plots from Dedalus output:

In [39]:
def plot_energy(dir_path):
    """
    Plot energy as a function of time.

    Parameters
    ----------
    dir_path: str, the path to the directory containing the HDF5 output generated by Dedalus.
    """
    ax = plt.axes(xlabel=r'$t$', ylabel='Energy', yscale='log')
    ax.grid(color='grey', ls='dotted')
    with h5py.File(f"{dir_path}/{dir_path}_s1.h5") as file:
        time = file['scales']['sim_time'][:]
        for label in ('KE1', 'KE2'):
            ax.plot(time, file['tasks'][label][:, 0, 0], label=label)
    ax.legend()  

In [3]:
def animation_streamfunction(dir_path, plot_full=False):
    """
    Plot an animation with four panels showing the streamfunction and potential vorticity in each layer.

    Parameters
    ----------
    dir_path: str, the path to the directory containing the HDF5 output generated by Dedalus.

    Keyword Arguments
    -----------------
    plot_full: bool, whether to show only the perturbation (default) or the full fields.
    """
    # Create figure with four panels
    fig, axes = plt.subplots(2, 2, figsize=(12.8, 9.6))

    # Read data from Dedalus output file
    with h5py.File(f"{dir_path}/{dir_path}_s1.h5") as file:
        psi1 = file['tasks']['psi1'][:]
        psi2 = file['tasks']['psi2'][:]
        q1 = file['tasks']['vorticity1'][:]
        q2 = file['tasks']['vorticity2'][:]
        time = file['scales']['sim_time'][:]
        params = dict(file.attrs)

    # Add background fields to perturbations if showing the full fields
    plot_type = "Perturbation"
    if plot_full:
        _, Nx, Ny = psi1.shape
        x = np.linspace(-np.pi, np.pi, Nx)
        y = np.linspace(-np.pi, np.pi, Ny)
        _, _, Y = np.meshgrid(x, time, y)
        psi1 = psi1 - params['mean_flow']*Y
        psi2 = psi2 + params['mean_flow']*Y
        q1 = q1 + (params['beta']+params['kd']**2*params['mean_flow'])*Y
        q2 = q2 + (params['beta']-params['kd']**2*params['mean_flow'])*Y
        plot_type = "Full field"
    fig.suptitle(f"{plot_type}, t=0")

    # Define auxiliary function to plot a field in a given panel
    def create_plot(ax, field, label):
        ax.set_xlabel(r"$x$")
        ax.set_ylabel(r"$y$")
        im = ax.imshow(field[0].T, extent=[-np.pi, np.pi, -np.pi, np.pi], cmap='RdBu_r', origin='lower')
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label(label)
        return im, cbar

    # Plot the four initial snapshots
    im_list, cbar_list = [], []
    for index, field, label in zip(range(4), (psi1, psi2, q1, q2), ('Streamfunction Top Layer', 'Streamfunction Bottom Layer', 'PV Top Layer', 'PV Bottom Layer')):
        ax = axes[int(index/2)][index%2]
        im, cbar = create_plot(ax, field, label)
        im_list.append(im)
        cbar_list.append(cbar)
    plt.tight_layout()
    plt.close()

    # Define auxiliary function to update the four panels
    def animate(i):
        fig.suptitle(f"{plot_type}, t={time[i]:.2f}")
        for im, cbar, field in zip(im_list, cbar_list, (psi1, psi2, q1, q2)):
            im.set_data(field[i].T)
            im.set_clim(np.min(field[i]), np.max(field[i]))  # Update colorbar limits dynamically
            cbar.update_normal(im)  # Update the colorbar to reflect the new image data
        return [im]

    # Create the animation using the above function
    ani = animation.FuncAnimation(fig, animate, frames=len(psi1), interval=50, blit=False)

    return ani

<div class="alert alert-warning">

In some cases, `matplotlib` might complain that the animation exceeds the allowed size. In that case you can run the following command to increase the allowed size:
```
plt.rc('animation', embed_limit=50)
```
to extend it to 50MB. The default value should be 20MB. You can check it with:
```
import matplotlib as mpl
mpl.rcParams['animation.embed_limit']
```

Alternatively, you can decrease the output frequency by adjusting the `snap_dt` parameter when running the simulation (remember that any machine has finite resources...).

</div>

# Rossby waves in the 2-layer model

We start by simulating the free modes of the 2-layer model in the absence of background flow ($U=0$), but with a gradient of planetary vorticity ($\beta \neq 0$).

Simulate the evolution of a small amplitude monochromatic perturbation $\psi_1'=\psi_0 e^{i(kx+\ell y)}$ in the upper layer, with two vertical structures: $\psi_2'=\psi_1'$ or $\psi_2'=-\psi_1'$.
In each case:
- plot an animation to check visually that the perturbation indeed propagates zonnally while preserving its space-time coherence.
- make a Hovmoller diagram to check that the phase speed agrees with theory.

For these simulations you do not need to add viscosity, you can set $\nu=0$.

Does the amplitude of the perturbation evolve in time in the absence of dissipation?
What is the sign of the background potential vorticity gradient in each layer?
How are these two things related?

<div class="alert alert-info">

*Note on parameter values*
    
Typical parameter values for the mid-latitude atmosphere or ocean would be $\beta\approx 1.5e^{-11}$ m$^{-1}$.s$^{-1}$.
    
Now, the equations above are written in dimensional form, but we choose a domain of size $2\pi$, which amounts at non-dimensionalizing the $x$ and $y$ coordinates by a scale $L$ (for a circle at latitude 45, for instance, the unit lenght $L$ is on the order of 4500km). Hence all the parameters should be written with the corresponding unit length. In addition, it is natural here to choose $(\beta{}L)^{-1}$ as the unit time. With this choice, the only remaining parameter is the deformation radius, which is on order 10 or so in this unit system. Hence in this section we should run simulations using the code above with $\beta=1$ and $k_d=10$.

</div>


To illustrate how to use the code, here is one way to run a simulation:

In [ ]:
problem = build_problem(beta=1, U=0, kd2=100, nu=0, alpha=0)

# Initial condition
problem.namespace['psi1']['g'] = monochromatic_wave(1, 0, ampl=0.01)
problem.namespace['psi2']['g'] = monochromatic_wave(1, 0, ampl=0.01)
problem.namespace['q1']['g'], problem.namespace['q2']['g'] = pv_from_streamfunction(problem.namespace['psi1'], problem.namespace['psi2'], problem.namespace['kd2'])

run_simulation("phillips_beta=1_U=0_kd2=100_ic_k1_l0_barotropic", problem, stop_sim_time=10, initial_timestep=1e-2)

# Baroclinic instability on the f-plane ($\beta=0$)

In the above section, we have seen that with beta effect but in the absence of vertical shear, the 2-layer QG model is stable and supports two westward propagating modes. Now we will study the opposite situation, with vertical shear ($U>0$) but no beta effect. This is the simplest setup for baroclinic instability.

Again rather than using dimensional values for $U$, we shall use $T=L/U$ as the time unit, which amounts at simply setting $U=1$ in the dimensional equations with $\beta=0$. We keep $k_{d}=10$.

- run a simulation with the following initial condition: $\psi_{1}'=\psi_0e^{i(kx+\ell{}y)}$, $\psi_{1}'=\psi_0e^{i(kx+\ell{}y+\pi/4)}$. Plot an animation of the evolution of the stream function and potential vorticity in the two layers, and the evolution in time of the kinetic energy in each layer. Identify the different stages of the simulation: in particular, do you observe a behavior compatible with the linear stability analysis of the Phillips problem?
  For this simulation you will need to add a small viscosity to avoid the build-up of enstrophy at the end of the exponential growth stage. With the hyperviscosity used in the code, you can choose $\nu=10^{-4}$.
- does the evolution depend on the initial condition?
- what is the meridional and zonal wavenumber of the unstable perturbation? Is this compatible with the linear theory? Why do you always observe this mode regardless of the initial condition?
- compare the stream function in the two layers in the linear instability regime: is there a phase difference? Compared to the background vertical shear, does it go in the same direction or in the opposite direction?

# Baroclinic instability with beta effect

We now look at the effect of differential rotation on the baroclinic instability.

Keeping the same value as above for all the other parameters, progressively increase $\beta$ and observe its effect on the dynamics. In particular:
- check that with increasing $\beta$, the wavelength of the most unstable mode and its growth rate both decrease.
- check that for $\beta$ large enough, the instability disappears. Why is that?
- describe the effect of $\beta$ on the shape of the equilibrated flow.

Note: in our unit system, we could estimate that typical values of $\beta$ for the atmosphere should be of order 10 and of order 100 for the ocean.